In [ ]:
from lxml import etree   # import etree functionality from lxml library
import urllib.request  # import urllib.request module for URL handling
import os  # import os module for OS interaction
import rdflib  # import rdflib library for RDF graph processing
import pyRdfa  # import pyRdfa library for RDFa parsing

In [2]:
def get_rdfa(xmlfile: str):  # define function with parameters
    
    # check XML file extension
    if not xmlfile.lower().endswith('.xml'):
        raise ValueError('Invalid file format.')

    # try loading XML file
    try:
        if xmlfile.startswith('http://') or xmlfile.startswith('https://'):  # check if XML file has URL source
            with urllib.request.urlopen(xmlfile) as response:  # send request to URL to open XML file
                xml_bytes = response.read()  # read URL raw byte content
                tree = etree.ElementTree(etree.fromstring(xml_bytes))  # parse raw bytes into XML element structure and wrap structure inside ElementTree
            xml_source = xmlfile  # pass XML file URL to variable
        elif os.path.isfile(xmlfile):  # check if XML file exists locally
            with open(xmlfile, 'rb') as xml_file:  # open XML file in binary mode
                tree = etree.parse(xml_file)  # parse XML file object directly into ElementTree
            xml_source = 'file://' + os.path.abspath(xmlfile)  # pass XML file URI to variable
        else:
            raise ValueError('Invalid file path or URL format.')

        # feedback successful XML parsing
        root = tree.getroot()  # retrieve XML-tree root element
        print(f'Successfully loaded {xmlfile}.\n'
            f'XML root element {root.tag} has direct child elements ' + ', '.join(element.tag for element in root) + '.')  # join list of tag names into comma-separated string
        
        # Parse RDFa content using pyRdfa
        # Create an empty rdflib graph to store the triples
        g = rdflib.Graph()
        
        # Use pyRdfa to extract RDFa and add to the graph
        rdfa_parser = pyRdfa.pyRdfa()
        result = rdfa_parser.graph_from_source(xml_source, graph=g)
        
        # Simple output of RDFa triples
        print(f"\nFound {len(result)} RDFa triples:")
        if len(result) > 0:
            for subject, predicate, obj in result:
                print(f"{subject} - {predicate} - {obj}")
        else:
            print("No RDFa content found in the XML file.")
        
        return result  # Return the RDFa graph for further processing

    except Exception as e:  # catch exception, store details
        raise OSError(f'Error loading XML file: {e}.')  # feedback exception details

In [3]:
get_rdfa('https://raw.githubusercontent.com/auden-in-austria-digital/aad-data/refs/heads/dev/data/xml/editions/aad-transcript__0032.xml')

Successfully loaded https://raw.githubusercontent.com/auden-in-austria-digital/aad-data/refs/heads/dev/data/xml/editions/aad-transcript__0032.xml.
XML root element {http://www.tei-c.org/ns/1.0}TEI has direct child elements {http://www.tei-c.org/ns/1.0}teiHeader, {http://www.tei-c.org/ns/1.0}facsimile, {http://www.tei-c.org/ns/1.0}text.


rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xml:base="https://id.acdh.oeaw.ac.at/auden-in-austria-digital" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.tei-c.org/ns/1.0"/ does not look like a valid URI, trying to serialize this will break.



Found 8 RDFa triples:
https://amp.acdh.oeaw.ac.at/amp_person_254.html - http://www.w3.org/2000/01/rdf-schema#label - Gäste
wd:Q62566483 - http://www.w3.org/2000/01/rdf-schema#label - Thekla Clark
https://amp.acdh.oeaw.ac.at/amp_person_254.html - crm:P107_has_current_or_former_member - wd:Q62566483
wd:Q123679692 - http://www.w3.org/2000/01/rdf-schema#label - John Clark
https://amp.acdh.oeaw.ac.at/amp_person_254.html - crm:P107_has_current_or_former_member - wd:Q123679692
wd:Q123679692 - http://www.w3.org/1999/02/22-rdf-syntax-ns#type - crm:E21_Person
wd:Q62566483 - http://www.w3.org/1999/02/22-rdf-syntax-ns#type - crm:E21_Person
https://amp.acdh.oeaw.ac.at/amp_person_254.html - http://www.w3.org/1999/02/22-rdf-syntax-ns#type - crm:E74_Group


<Graph identifier=N7839e193007d49b1a020bb315a1643f1 (<class 'rdflib.graph.Graph'>)>